<a href="https://colab.research.google.com/github/fgsantosti/ProgramacaoDispositivosMoveisFlutter/blob/main/CRUD_usando_Flutter_e_SQLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CRUD usando Flutter e SQLite

Adicionando os importes no arquivo ```pubspec.yaml```

In [ ]:
  sqflite: ^2.0.2
  path_provider: ^2.0.9

Sistema de pastas do nosso app

In [ ]:
* lib
 	  * db_helper
		  - database_conexao.dart
		  - repositorio.dart 
		* model
			- usuario.dart
		* screens
			- add_usuario.dart
			- editar_usuario.dart
			- listar_usuarios.dart
		* services
			- usuario_service.dart
  main.dart

Arquivo ```usuario.dart```

In [ ]:
class User {
  int? id;
  String? nome;
  String? contato;
  String? descricao;
  userMap() {
    var mapping = Map<String, dynamic>();
    mapping['id'] = id ?? null;
    mapping['nome'] = nome!;
    mapping['contato'] = contato!;
    mapping['descricao'] = descricao!;
    return mapping;
  }
}

Arquivo ```database_conexao.dart```

In [ ]:
import 'package:sqflite/sqflite.dart';
import 'package:path/path.dart';
import 'package:path_provider/path_provider.dart';

class DatabaseConnection {
  Future<Database> setDatabase() async {
    var directory = await getApplicationDocumentsDirectory();
    var path = join(directory.path, 'db_crud');
    var database =
        await openDatabase(path, version: 1, onCreate: _createDatabase);
    return database;
  }

  Future<void> _createDatabase(Database database, int version) async {
    String sql =
        "CREATE TABLE usuarios (id INTEGER PRIMARY KEY,nome TEXT,contato TEXT,descricao TEXT);";
    await database.execute(sql);
  }
}


Arquivo ```repositorio.dart```

In [ ]:
import 'database_conexao.dart';
import 'package:sqflite/sqflite.dart';

class Repository {
  late DatabaseConnection _databaseConnection;
  Repository() {
    _databaseConnection = DatabaseConnection();
  }
  static Database? _database;
  Future<Database?> get database async {
    if (_database != null) {
      return _database;
    } else {
      _database = await _databaseConnection.setDatabase();
      return _database;
    }
  }

  insertData(table, data) async {
    var conexao = await database;
    return await conexao?.insert(table, data);
  }

  readData(table) async {
    var conexao = await database;
    return await conexao?.query(table);
  }

  readDataById(table, itemId) async {
    var conexao = await database;
    return await conexao?.query(table, where: 'id=?', whereArgs: [itemId]);
  }

  updateData(table, data) async {
    var conexao = await database;
    return await conexao
        ?.update(table, data, where: 'id=?', whereArgs: [data['id']]);
  }

  deleteDataById(table, itemId) async {
    var conexao = await database;
    return await conexao?.rawDelete("delete from $table where id=$itemId");
  }
}

Arquivo ```usuario_service.dart```

In [ ]:
import '../db_helper/repositorio.dart';
import '../model/usuario.dart';

class UserService {
  late Repository _repository;
  UserService() {
    _repository = Repository();
  }
  //Save User
  SaveUser(User user) async {
    return await _repository.insertData('usuarios', user.userMap());
  }

  //Read All Users
  readAllUsers() async {
    return await _repository.readData('usuarios');
  }

  //Edit User
  UpdateUser(User user) async {
    return await _repository.updateData('usuarios', user.userMap());
  }

  deleteUser(userId) async {
    return await _repository.deleteDataById('usuarios', userId);
  }
}


Arquivo ```add_usuario.dart```

In [ ]:
import 'package:widget_basicos/model/usuario.dart';
import 'package:flutter/material.dart';

import '../services/usuario_service.dart';

class AddUser extends StatefulWidget {
  const AddUser({Key? key}) : super(key: key);

  @override
  State<AddUser> createState() => _AddUserState();
}

class _AddUserState extends State<AddUser> {
  var _userNomeController = TextEditingController();
  var _userContatoController = TextEditingController();
  var _userDescricaoController = TextEditingController();
  bool _validateNome = false;
  bool _validateContato = false;
  bool _validateDescricao = false;
  var _userService = UserService();
  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text("SQLite CRUD"),
      ),
      body: SingleChildScrollView(
        child: Container(
          padding: const EdgeInsets.all(16.0),
          child: Column(
            crossAxisAlignment: CrossAxisAlignment.start,
            children: [
              const Text(
                'Adicionar novo usuário',
                style: TextStyle(
                    fontSize: 20,
                    color: Colors.teal,
                    fontWeight: FontWeight.w500),
              ),
              const SizedBox(
                height: 20.0,
              ),
              TextField(
                  controller: _userNomeController,
                  decoration: InputDecoration(
                    border: const OutlineInputBorder(),
                    hintText: 'Nome',
                    labelText: 'Nome',
                    errorText: _validateNome
                        ? 'O valor do nome não pode estar vazio'
                        : null,
                  )),
              const SizedBox(
                height: 20.0,
              ),
              TextField(
                  controller: _userContatoController,
                  decoration: InputDecoration(
                    border: const OutlineInputBorder(),
                    hintText: 'Contato',
                    labelText: 'Contato',
                    errorText: _validateContato
                        ? 'O valor do contato não pode estar vazio'
                        : null,
                  )),
              const SizedBox(
                height: 20.0,
              ),
              TextField(
                  controller: _userDescricaoController,
                  decoration: InputDecoration(
                    border: const OutlineInputBorder(),
                    hintText: 'Descrição',
                    labelText: 'Descrição',
                    errorText: _validateDescricao
                        ? 'O valor do descrição não pode estar vazio'
                        : null,
                  )),
              const SizedBox(
                height: 20.0,
              ),
              Row(
                children: [
                  TextButton(
                      style: TextButton.styleFrom(
                          primary: Colors.white,
                          backgroundColor: Colors.teal,
                          textStyle: const TextStyle(fontSize: 15)),
                      onPressed: () async {
                        setState(() {
                          _userNomeController.text.isEmpty
                              ? _validateNome = true
                              : _validateNome = false;
                          _userContatoController.text.isEmpty
                              ? _validateContato = true
                              : _validateContato = false;
                          _userDescricaoController.text.isEmpty
                              ? _validateDescricao = true
                              : _validateDescricao = false;
                        });
                        if (_validateNome == false &&
                            _validateContato == false &&
                            _validateDescricao == false) {
                          // print("Good Data Can Save");
                          var _user = User();
                          _user.nome = _userNomeController.text;
                          _user.contato = _userContatoController.text;
                          _user.descricao = _userDescricaoController.text;
                          var result = await _userService.SaveUser(_user);
                          Navigator.pop(context, result);
                        }
                      },
                      child: const Text('Dados Salvos')),
                  const SizedBox(
                    width: 10.0,
                  ),
                  TextButton(
                      style: TextButton.styleFrom(
                          primary: Colors.white,
                          backgroundColor: Colors.red,
                          textStyle: const TextStyle(fontSize: 15)),
                      onPressed: () {
                        _userNomeController.text = '';
                        _userContatoController.text = '';
                        _userDescricaoController.text = '';
                      },
                      child: const Text('Limpar Dados'))
                ],
              )
            ],
          ),
        ),
      ),
    );
  }
}


Arquivo ```editar_usuario.dart```

In [ ]:
import 'package:flutter/material.dart';
import '../model/usuario.dart';
import '../services/usuario_service.dart';

class EditUser extends StatefulWidget {
  final User user;
  const EditUser({Key? key, required this.user}) : super(key: key);

  @override
  State<EditUser> createState() => _EditUserState();
}

class _EditUserState extends State<EditUser> {
  var _userNomeController = TextEditingController();
  var _userContatoController = TextEditingController();
  var _userDescricaoController = TextEditingController();
  bool _validateNome = false;
  bool _validateContato = false;
  bool _validateDescricao = false;
  var _userService = UserService();

  @override
  void initState() {
    setState(() {
      _userNomeController.text = widget.user.nome ?? '';
      _userContatoController.text = widget.user.contato ?? '';
      _userDescricaoController.text = widget.user.descricao ?? '';
    });
    super.initState();
  }

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text("SQLite CRUD"),
      ),
      body: SingleChildScrollView(
        child: Container(
          padding: const EdgeInsets.all(16.0),
          child: Column(
            crossAxisAlignment: CrossAxisAlignment.start,
            children: [
              const Text(
                'Editar Usuário',
                style: TextStyle(
                    fontSize: 20,
                    color: Colors.teal,
                    fontWeight: FontWeight.w500),
              ),
              const SizedBox(
                height: 20.0,
              ),
              TextField(
                  controller: _userNomeController,
                  decoration: InputDecoration(
                    border: const OutlineInputBorder(),
                    hintText: 'Nome',
                    labelText: 'Nome',
                    errorText: _validateNome
                        ? 'O valor do nome não pode estar vazio'
                        : null,
                  )),
              const SizedBox(
                height: 20.0,
              ),
              TextField(
                  controller: _userContatoController,
                  decoration: InputDecoration(
                    border: const OutlineInputBorder(),
                    hintText: 'Contato',
                    labelText: 'Contato',
                    errorText: _validateContato
                        ? 'O valor do contato não pode estar vazio'
                        : null,
                  )),
              const SizedBox(
                height: 20.0,
              ),
              TextField(
                  controller: _userDescricaoController,
                  decoration: InputDecoration(
                    border: const OutlineInputBorder(),
                    hintText: 'Descrição',
                    labelText: 'Descrição',
                    errorText: _validateDescricao
                        ? 'O valor do descrição não pode estar vazio'
                        : null,
                  )),
              const SizedBox(
                height: 20.0,
              ),
              Row(
                children: [
                  TextButton(
                      style: TextButton.styleFrom(
                          primary: Colors.white,
                          backgroundColor: Colors.teal,
                          textStyle: const TextStyle(fontSize: 15)),
                      onPressed: () async {
                        setState(() {
                          _userNomeController.text.isEmpty
                              ? _validateNome = true
                              : _validateNome = false;
                          _userContatoController.text.isEmpty
                              ? _validateContato = true
                              : _validateContato = false;
                          _userDescricaoController.text.isEmpty
                              ? _validateDescricao = true
                              : _validateDescricao = false;
                        });
                        if (_validateNome == false &&
                            _validateContato == false &&
                            _validateDescricao == false) {
                          // print("Good Data Can Save");
                          var _user = User();
                          _user.id = widget.user.id;
                          _user.nome = _userNomeController.text;
                          _user.contato = _userContatoController.text;
                          _user.descricao = _userDescricaoController.text;
                          var result = await _userService.UpdateUser(_user);
                          Navigator.pop(context, result);
                        }
                      },
                      child: const Text('Dados Atualizados')),
                  const SizedBox(
                    width: 10.0,
                  ),
                  TextButton(
                      style: TextButton.styleFrom(
                          primary: Colors.white,
                          backgroundColor: Colors.red,
                          textStyle: const TextStyle(fontSize: 15)),
                      onPressed: () {
                        _userNomeController.text = '';
                        _userContatoController.text = '';
                        _userDescricaoController.text = '';
                      },
                      child: const Text('Limpar Campos'))
                ],
              )
            ],
          ),
        ),
      ),
    );
  }
}


Arquivo ```listar_usuario.dart```

In [ ]:
import 'package:flutter/material.dart';
import '../model/usuario.dart';

class ViewUser extends StatefulWidget {
  final User user;

  const ViewUser({Key? key, required this.user}) : super(key: key);

  @override
  State<ViewUser> createState() => _ViewUserState();
}

class _ViewUserState extends State<ViewUser> {
  @override
  Widget build(BuildContext context) {
    return Scaffold(
        appBar: AppBar(
          title: const Text("SQLite CRUD"),
        ),
        body: Container(
          padding: EdgeInsets.all(16.0),
          child: Column(
            crossAxisAlignment: CrossAxisAlignment.start,
            children: [
              const Text(
                "Full Details",
                style: TextStyle(
                    fontWeight: FontWeight.w600,
                    color: Colors.blueGrey,
                    fontSize: 20),
              ),
              const SizedBox(
                height: 20,
              ),
              Row(
                children: [
                  const Text('Nome',
                      style: TextStyle(
                          color: Colors.teal,
                          fontSize: 16,
                          fontWeight: FontWeight.w600)),
                  Padding(
                    padding: const EdgeInsets.only(left: 30),
                    child: Text(widget.user.nome ?? '',
                        style: TextStyle(fontSize: 16)),
                  ),
                ],
              ),
              const SizedBox(
                height: 20,
              ),
              Row(
                children: [
                  const Text('Contato',
                      style: TextStyle(
                          color: Colors.teal,
                          fontSize: 16,
                          fontWeight: FontWeight.w600)),
                  Padding(
                    padding: const EdgeInsets.only(left: 25),
                    child: Text(widget.user.contato ?? '',
                        style: TextStyle(fontSize: 16)),
                  ),
                ],
              ),
              const SizedBox(
                height: 20,
              ),
              Column(
                crossAxisAlignment: CrossAxisAlignment.start,
                children: [
                  const Text('Descrição',
                      style: TextStyle(
                          color: Colors.teal,
                          fontSize: 16,
                          fontWeight: FontWeight.w600)),
                  const SizedBox(
                    height: 20,
                  ),
                  Text(widget.user.descricao ?? '',
                      style: const TextStyle(fontSize: 16)),
                ],
              )
            ],
          ),
        ));
  }
}


Arquivo ```main.dart```

In [ ]:
import 'package:flutter/material.dart';

import 'model/usuario.dart';
import 'screens/add_usuario.dart';
import 'screens/editar_usuario.dart';
import 'screens/listar_usuarios.dart';
import 'services/usuario_service.dart';

void main() {
  runApp(const MyApp());
}

class MyApp extends StatelessWidget {
  const MyApp({Key? key}) : super(key: key);

  @override
  Widget build(BuildContext context) {
    return MaterialApp(
      title: 'Flutter Demo',
      debugShowCheckedModeBanner: false,
      theme: ThemeData(
        primarySwatch: Colors.teal,
      ),
      home: const MyHomePage(),
    );
  }
}

class MyHomePage extends StatefulWidget {
  const MyHomePage({Key? key}) : super(key: key);

  @override
  State<MyHomePage> createState() => _MyHomePageState();
}

class _MyHomePageState extends State<MyHomePage> {
  late List<User> _userList = <User>[];
  final _userService = UserService();

  getAllUserDetails() async {
    var users = await _userService.readAllUsers();
    _userList = <User>[];
    users.forEach((user) {
      setState(() {
        var userModel = User();
        userModel.id = user['id'];
        userModel.nome = user['nome'];
        userModel.contato = user['contato'];
        userModel.descricao = user['descricao'];
        _userList.add(userModel);
      });
    });
  }

  @override
  void initState() {
    getAllUserDetails();
    super.initState();
  }

  _showSuccessSnackBar(String message) {
    ScaffoldMessenger.of(context).showSnackBar(
      SnackBar(
        content: Text(message),
      ),
    );
  }

  _deleteFormDialog(BuildContext context, userId) {
    return showDialog(
        context: context,
        builder: (param) {
          return AlertDialog(
            title: const Text(
              'Tem certeza que deseja excluir',
              style: TextStyle(color: Colors.teal, fontSize: 20),
            ),
            actions: [
              TextButton(
                  style: TextButton.styleFrom(
                      primary: Colors.white, // foreground
                      backgroundColor: Colors.red),
                  onPressed: () async {
                    var result = await _userService.deleteUser(userId);
                    if (result != null) {
                      Navigator.pop(context);
                      getAllUserDetails();
                      _showSuccessSnackBar('Usuário Deletado com Sucesso');
                    }
                  },
                  child: const Text('Deletar')),
              TextButton(
                  style: TextButton.styleFrom(
                      primary: Colors.white, // foreground
                      backgroundColor: Colors.teal),
                  onPressed: () {
                    Navigator.pop(context);
                  },
                  child: const Text('Fechar'))
            ],
          );
        });
  }

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text("SQLite CRUD"),
      ),
      body: ListView.builder(
          itemCount: _userList.length,
          itemBuilder: (context, index) {
            return Card(
              child: ListTile(
                onTap: () {
                  Navigator.push(
                      context,
                      MaterialPageRoute(
                          builder: (context) => ViewUser(
                                user: _userList[index],
                              )));
                },
                leading: const Icon(Icons.person),
                title: Text(_userList[index].nome ?? ''),
                subtitle: Text(_userList[index].contato ?? ''),
                trailing: Row(
                  mainAxisSize: MainAxisSize.min,
                  children: [
                    IconButton(
                        onPressed: () {
                          Navigator.push(
                              context,
                              MaterialPageRoute(
                                  builder: (context) => EditUser(
                                        user: _userList[index],
                                      ))).then((data) {
                            if (data != null) {
                              getAllUserDetails();
                              _showSuccessSnackBar(
                                  'Usuário Alterado com Sucesso');
                            }
                          });
                          ;
                        },
                        icon: const Icon(
                          Icons.edit,
                          color: Colors.teal,
                        )),
                    IconButton(
                        onPressed: () {
                          _deleteFormDialog(context, _userList[index].id);
                        },
                        icon: const Icon(
                          Icons.delete,
                          color: Colors.red,
                        ))
                  ],
                ),
              ),
            );
          }),
      floatingActionButton: FloatingActionButton(
        onPressed: () {
          Navigator.push(context,
                  MaterialPageRoute(builder: (context) => const AddUser()))
              .then((data) {
            if (data != null) {
              getAllUserDetails();
              _showSuccessSnackBar('Usuário Adicionado com Sucesso');
            }
          });
        },
        child: const Icon(Icons.add),
      ),
    );
  }
}
